# speech_to_text 语音转文本

使用PyAudio，录制音频后，通过OpenAI的Speech To Text接口转换为文本

In [28]:
# 相关的安装包
!pip install PyAudio
!pip install openai
!pip install transformers
!pip install langchain
!pip install torch
!pip install pillow

In [26]:
import pyaudio
import wave
import sys
import time
import os

CHUNK = 1024  # 每个缓冲区的帧数
FORMAT = pyaudio.paInt16  # 采样位数
CHANNELS = 1  # 单声道
RATE = 44100  # 采样频率


def record_audio(wave_out_path, record_second):
    """ 录音功能 """
    p = pyaudio.PyAudio()  # 实例化对象
    if os.path.exists(wave_out_path):
        print("移除文件")
        os.remove(wave_out_path)
    print(f'ready to recording | {wave_out_path}....')
    for i in range(3):
        print(f"{3-i}")
        time.sleep(1)
        
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)  # 打开流，传入响应参数
    wf = wave.open(wave_out_path, 'wb')  # 打开 wav 文件。
    print('start recording....')
    wf.setnchannels(CHANNELS)  # 声道设置
    wf.setsampwidth(p.get_sample_size(FORMAT))  # 采样位数设置
    wf.setframerate(RATE)  # 采样频率设置

    for _ in range(0, int(RATE * record_second / CHUNK)):
        data = stream.read(CHUNK)
        wf.writeframes(data)  # 写入数据
    stream.stop_stream()  # 关闭流
    print('end recording....')
    stream.close()
    p.terminate()
    wf.close()


def play_audio(wave_input_path):
    p = pyaudio.PyAudio()  # 实例化
    wf = wave.open(wave_input_path, 'rb')  # 读 wav 文件
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)
    data = wf.readframes(CHUNK)  # 读数据
    while len(data) > 0:
        stream.write(data)
        data = wf.readframes(CHUNK)

    stream.stop_stream()  # 关闭资源
    stream.close()
    p.terminate()

In [ ]:
# 录制5秒的音频
record_audio('./test.mp3', 5)

In [27]:
# 使用openai接口将音频转为文字
from openai import OpenAI
client = OpenAI()

# 需要在环境变量里配置openai的apikey
audio_file= open("./test.mp3", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)
print(transcription.text)

I record my first videos here, before AI, before Academy.
